## Data Cleaning

First import necessary packages for cleaning and anything else that is needed.

In [1]:
import pandas as pd
import os
from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

Loading data into dataframes

In [2]:
Bommoviegross = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')
BomGross = pd.DataFrame(Bommoviegross)

imdbname = pd.read_csv('./zippedData/imdb.name.basics.csv.gz')
imdbName = pd.DataFrame(imdbname)

imdbtitlea = pd.read_csv('./zippedData/imdb.title.akas.csv.gz')
imdbTitleAkas = pd.DataFrame(imdbtitlea)

imdbtitleb = pd.read_csv('./zippedData/imdb.title.basics.csv.gz')
imdbTitleBasic = pd.DataFrame(imdbtitleb)

imdbtitlec = pd.read_csv('./zippedData/imdb.title.crew.csv.gz')
imdbTitleCrew = pd.DataFrame(imdbtitlec)

imdbtitlep = pd.read_csv('./zippedData/imdb.title.principals.csv.gz')
imdbTitlePrin = pd.DataFrame(imdbtitlep)

imdbtitler = pd.read_csv('./zippedData/imdb.title.ratings.csv.gz')
imdbTitleRat = pd.DataFrame(imdbtitler)

rtmov = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep='\t')
rtmovie = pd.DataFrame(rtmov)

rtrev = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep='\t', encoding= 'unicode_escape')
rtreview = pd.DataFrame(rtrev)

tmdb = pd.read_csv('./zippedData/tmdb.movies.csv.gz')
tmdbmov = pd.DataFrame(rtreview)

tnm = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz')
tnmmov = pd.DataFrame(tnm)

Start looking at the data frames to see what kind of data we have.

In [61]:
#Joining imdb tables to make a master table
q = '''SELECT *
       FROM imdbTitleBasic
       LEFT JOIN imdbTitleRat
       USING (tconst)
       Left JOIN imdbTitleCrew
       USING (tconst)
       ;
    '''
imdbMaster = pysqldf(q)

#cleaning up directors and writers columns, putting in the actual names
imdbMaster['directors'] = imdbMaster['directors'].astype(str)
imdbMaster['writers'] = imdbMaster['writers'].astype(str)
imdbMaster['directors'] = imdbMaster['directors'].map(lambda x: x.split(','))
imdbMaster['writers'] = imdbMaster['writers'].map(lambda x: x.split(','))
All_names = imdbName.set_index('nconst')['primary_name'].to_dict()
DirectorNamesList = []
for entry in imdbMaster['directors']:
    DirectorNamesList.append([All_names.get(item,item) for item in entry])
imdbMaster['Director_Names'] = DirectorNamesList
WritersNamesList = []
for entry in imdbMaster['writers']:
    WritersNamesList.append([All_names.get(item,item) for item in entry])
imdbMaster['Writers_Names'] = WritersNamesList

In [ ]:
imdbMaster['Director_Names'].nunique()

In [62]:
imdbMaster.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,directors,writers,Director_Names,Writers_Names
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,[nm0712540],"[nm0023551, nm1194313, nm0347899, nm1391276]",[Harnam Singh Rawail],"[Abrar Alvi, Mahasweta Devi, Gulzar, Anjana Ra..."
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,[nm0002411],[None],[Mani Kaul],[None]
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,[nm0000080],"[nm0000080, nm0462648]",[Orson Welles],"[Orson Welles, Oja Kodar]"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,[nm0611531],[nm0347899],[Hrishikesh Mukherjee],[Gulzar]
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"[nm0765384, nm0749914]","[nm1360635, nm0749914]","[Valeria Sarmiento, Raoul Ruiz]","[Pía Rey, Raoul Ruiz]"


In [56]:
q = '''SELECT m.*, n.primary_name AS DirectorName
       FROM imdbMaster m
       LEFT JOIN imdbName n
       ON m.directors = n.nconst
       ;
    '''
test = sqldf(q)
test.head()

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 8 - probably unsupported type.
[SQL: INSERT INTO "imdbMaster" (tconst, primary_title, original_title, start_year, runtime_minutes, genres, averagerating, numvotes, directors, writers) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('tt0063540', 'Sunghursh', 'Sunghursh', 2013, 175.0, 'Action,Crime,Drama', 7.0, 77.0, ['nm0712540'], 'nm0023551,nm1194313,nm0347899,nm1391276'), ('tt0066787', 'One Day Before the Rainy Season', 'Ashad Ka Ek Din', 2019, 114.0, 'Biography,Drama', 7.2, 43.0, ['nm0002411'], None), ('tt0069049', 'The Other Side of the Wind', 'The Other Side of the Wind', 2018, 122.0, 'Drama', 6.9, 4517.0, ['nm0000080'], 'nm0000080,nm0462648'), ('tt0069204', 'Sabse Bada Sukh', 'Sabse Bada Sukh', 2018, None, 'Comedy,Drama', 6.1, 13.0, ['nm0611531'], 'nm0347899'), ('tt0100275', 'The Wandering Soap Opera', 'La Telenovela Errante', 2017, 80.0, 'Comedy,Drama,Fantasy', 6.5, 119.0, ['nm0765384', 'nm0749914'], 'nm136063

In [32]:
imdbMaster.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,directors,writers
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,"[nm0712540, [Harnam Singh Rawail]]","[nm0023551, nm1194313, nm0347899, nm1391276]"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,"[nm0002411, [Mani Kaul]]",[None]
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0,"[nm0000080, [Orson Welles]]","[nm0000080, nm0462648]"
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0,"[nm0611531, [Hrishikesh Mukherjee]]",[nm0347899]
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0,"[nm0765384, nm0749914, [Valeria Sarmiento, Rao...","[nm1360635, nm0749914]"
